In [ ]:
import csv
import requests
from bs4 import BeautifulSoup

In [ ]:
apollo_responses = dict()

for keyword in keyword_site_urls:
    if 'apollopharmacy' in keyword_site_urls[keyword].keys():
        url = keyword_site_urls[keyword]['apollopharmacy']
        # Sending a GET request to the URL
        response = requests.get(url)
        # Checking if the request was successful
        if response.status_code == 200:
            apollo_responses[keyword] = response.text
            print("HTML content fetched and saved.")
        else:
            print(f"Failed to fetch webpage. Status code: {response.status_code}")


In [ ]:
apollo_data = list()
apollo_dict = dict()
for keyword in apollo_responses:
    print (keyword)
    
    # try:
    soup = BeautifulSoup(apollo_responses[keyword], 'html.parser')

    # images_div =  soup.find('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_pdpGallery')).find_all('img')
    #images = ''
    # for image in images_div:
    #     print (image)
    #     src_value = image['src'] if image else None
    #     images += src_value + '\n'
    
    drug_title = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_title'))[0].get_text()
    prescription_drug = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_pdpTagWrapper'))[0].get_text()
    product_info_div = soup.find('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_productInfoRoot'))
    product_info_grids = soup.find_all('div', class_=lambda value: value and value.startswith('Grid_Item'))
    manufacturer = product_info_grids[1].get_text()
    
    product_data = soup.find_all('div', class_='yh')
    product_info_blocks = product_data[0].find_all('div', class_='RP')
    about = ''
    uses = ''
    benefits = ''
    directions = ''
    storage = ''
    side_effects = ''
    for pi_block in product_info_blocks:
        title = pi_block.find('h2').get_text()
        desc = '\n'.join(para.get_text() for para in pi_block.find_all('p'))
        if not desc:
            desc = '\n'.join(para.get_text() for para in pi_block.find_all('li'))
        if not desc:
            desc = pi_block.find('div', class_='VP').get_text()
        if 'About' in title:
            about = desc
        elif 'Uses' in title:
            uses = desc
        elif 'Benefits' in title:
            benefits = desc
        elif 'Directions' in title:
            directions = desc
        elif 'Storage' in title:
            storage = desc
        elif 'Side Effects' in title:
            side_effects = desc
    
    try:
        in_depth_info_blocks = product_data[1].find_all('div', class_='RP')
        warnings = ''
        interactions_checkers = ''
        interactions = ''
        habit = ''
        diet = ''
        advise = ''
        for pi_block in in_depth_info_blocks:
            title = pi_block.find('h2').get_text()
            desc = '\n'.join(para.get_text() for para in pi_block.find_all('p'))
            if not desc:
                desc = '\n'.join(para.get_text() for para in pi_block.find_all('li'))
            if not desc:
                desc = pi_block.find('div', class_='VP').get_text()
            if 'Warnings' in title:
                warnings = desc
            elif 'Checker' in title:
                interactions_checkers = desc
            elif 'Interactions' in title:
                interactions = desc
            elif 'Habit' in title:
                habit = desc
            elif 'Diet' in title:
                diet = desc
            elif 'Advise' in title:
                advise = desc
                
    except:
        in_depth_info = ''
    try:
        patients_concern = product_data[2].get_text()
    except:
        patients_concern = ''

    try:
        safety_text = ''
        safety_data = soup.find('div', class_='bb').find_all('div', class_="W_")
        for row in safety_data:
            safety_title = row.find('p', class_='Ld').get_text()
            safety_tag = row.find('p', class_='_b').get_text()
            safety_detail = row.find_all('p', class_='Ld')[1].get_text()
            safety_text += 'Title:' + safety_title + '\n' + 'Tag:' + safety_tag + '\n' + 'Detail:' + safety_detail + '\n\n'
        
        print (safety_text)
    except :
        safety_text = ''
    
    faqs_text = ''
    faqs_div = soup.find_all('div', class_='a')
    for faq_div in faqs_div:
        ques = faq_div.find('div', class_='b').get_text()
        ans = faq_div.find('div', class_='g').get_text()
        faqs_text += ques + '\n' + ans + '\n\n'
    
    manufacturer_details = soup.find_all('div', class_='pJ')

    apollo_row = {
        'images': images,
        'drug_title': drug_title,
        'prescription_drug': prescription_drug,
        'product_info_div': product_info_div,
        'product_info_grids': product_info_grids,
        'manufacturer': manufacturer,
        'about': about,
        'uses': uses,
        'benefits': benefits,
        'directions': directions,
        'storage': storage,
        'side_effects': side_effects,
        'warnings': warnings,
        'interactions': interactions,
        'interactions_checkers': interactions_checkers,
        'habit': habit,
        'diet': diet,
        'advise': advise,
        'patients_concern': patients_concern,
        'safety_text': safety_text,
        'faqs_text': faqs_text,
        'manufacturer_details': manufacturer_details
        
    }
    apollo_data.append(apollo_row)
    apollo_dict[keyword] = apollo_row
    # break
    
f = open('/tmp/apollo.csv', 'w')
writer = DictWriter(f, fieldnames = apollo_data[0].keys())
writer.writeheader()
writer.writerows(apollo_data)
